In [ ]:
#| default_exp models.cnn
#| default_cls_lvl 3

In [ ]:
#| export
from tsfast.data import *
from tsfast.models.layers import *
from tsfast.models.rnn import *
from tsfast.learner.callbacks import *
from tsfast.learner.losses import *
from fastai.basics import *
from torch.nn.utils.parametrizations import weight_norm

In [ ]:
from tsfast.datasets import create_dls_test

In [ ]:
dls = create_dls_test()

## CNN

In [ ]:
#| export
@delegates(nn.Conv1d, keep=True)
def Conv1D(input_size,output_size,kernel_size=3,activation = Mish,wn=True, bn = False, **kwargs):
    conv = nn.Conv1d(input_size,output_size,kernel_size,**kwargs)
    act = activation() if activation is not None else None
    bn = nn.BatchNorm1d(input_size) if bn else None
    m = [m for m in [bn,conv,act] if m is not None]
    return nn.Sequential(*m)

In [ ]:
#| export
class CNN(nn.Module):
    def __init__(self,input_size,output_size,hl_depth=1,hl_width=10,act = Mish,bn=False):
        super().__init__()
        
        conv_layers = [Conv1D(input_size if i==0 else hl_width,hl_width,bn=bn,activation=act,padding=1)
                                          for i in range(hl_depth)]
        self.conv_layers = nn.Sequential(*conv_layers)
        self.final = nn.Conv1d(hl_width,output_size,kernel_size=1)
        
    def forward(self, x):      
        x_in = x.transpose(1,2)
        out = self.conv_layers(x_in)
        out = self.final(out).transpose(1,2)
        return out

In [ ]:
model = CNN(1,1,hl_depth=3)
lrn = Learner(dls,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,0.143014,0.124070,00:01


## TCN

In [ ]:
#| export
class CausalConv1d(torch.nn.Conv1d):
    def __init__(self,
                 in_channels,
                 out_channels,
                 kernel_size,
                 stride=1,
                 dilation=1,
                 groups=1,
                 bias=True,
                stateful=False):

        super().__init__(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=0,
            dilation=dilation,
            groups=groups,
            bias=bias)
        self.__init_size = (kernel_size - 1) * dilation
        self.x_init = None
        self.stateful = stateful
        
    def forward(self, x):
        if self.x_init is not None and self.x_init.shape[0] != x.shape[0]: 
            self.x_init = None
        
        if self.x_init is None or not self.stateful:
            self.x_init = torch.zeros((x.shape[0],x.shape[1],self.__init_size),device=x.device)
            
        x = torch.cat([self.x_init,x],dim=-1)
        
        out = super().forward(x)
    
        if self.stateful: self.x_init =to_detach( x[...,-self.__init_size:], cpu=False, gather=False) 
            
        return out
    
    def reset_state(self):
        self.x_init = None

In [ ]:
#| export
@delegates(CausalConv1d, keep=True)
def CConv1D(input_size,output_size,kernel_size=2,activation = Mish,wn=True, bn = False, **kwargs):
    conv = CausalConv1d(input_size,output_size,kernel_size,**kwargs)
    if wn: conv = weight_norm(conv)
    act = activation() if activation is not None else None
    bn = nn.BatchNorm1d(input_size) if bn else None
    m = [m for m in [bn,conv,act] if m is not None]
    return nn.Sequential(*m)

In [ ]:
#| export
@delegates(CausalConv1d, keep=True)
class TCN_Block(nn.Module):
    def __init__(self,input_size,output_size,num_layers=1,
                 activation = Mish,wn=True, bn = False,stateful=False, **kwargs):
        super().__init__()
        
        layers=[]
        for _ in range(num_layers):
            conv = CausalConv1d(input_size,output_size,2,stateful=stateful,**kwargs)
            if wn: conv = weight_norm(conv)
            act = activation() if activation is not None else None
            bn = nn.BatchNorm1d(input_size) if bn else None
            layers += [m for m in [bn,conv,act] if m is not None]
            
        self.layers = nn.Sequential(*layers)
        
        self.residual = nn.Conv1d(input_size,output_size,kernel_size=1) if output_size!=input_size else None
        
    def forward(self, x):      
        out = self.layers(x)
        out = out + (x if self.residual is None else self.residual(x))  
        return out

In [ ]:
#| export
class TCN(nn.Module):
    def __init__(self,input_size,output_size,hl_depth=1,hl_width=10,act = Mish,bn=False,stateful=False):
        super().__init__()
        
        conv_layers = [TCN_Block(input_size if i==0 else hl_width,hl_width,
                                      dilation=2**(i),bn=bn,activation=act,stateful=stateful)
                                          for i in range(hl_depth)]
        self.conv_layers = nn.Sequential(*conv_layers)
        self.final = nn.Conv1d(hl_width,output_size,kernel_size=1)
        
    def forward(self, x):      
        x_in = x.transpose(1,2)
        out = self.conv_layers(x_in)
        out = self.final(out).transpose(1,2)
        return out

In [ ]:
model = TCN(1,1,hl_depth=3)
lrn = Learner(dls,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,0.077889,0.072267,00:01


### TCN Learner
Performs better on multi input data. Higher beta values allow a way smoother prediction. Way faster then RNNs in prediction.


In [ ]:
#| export
from tsfast.data.loader import get_inp_out_size


In [ ]:
#| export
@delegates(TCN, keep=True)
def TCNLearner(dls,num_layers=3,hidden_size=100,loss_func=nn.L1Loss(),metrics=[fun_rmse],n_skip=None,opt_func=Adam,cbs=None,**kwargs):
    inp,out = get_inp_out_size(dls)
    n_skip = 2**num_layers if n_skip is None else n_skip
    model = TCN(inp,out,num_layers,hidden_size,**kwargs)
  
    skip = partial(SkipNLoss,n_skip=n_skip)
        
    metrics= [skip(f) for f in metrics]
    loss_func = skip(loss_func)
        
    lrn = Learner(dls,model,loss_func=loss_func,opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn


In [ ]:
from tsfast.datasets.core import create_dls_test
dls_pred = create_dls_test(prediction=True).cpu()
TCNLearner(dls_pred,num_layers=6,loss_func=nn.L1Loss()).fit(1)


epoch,train_loss,valid_loss,fun_rmse,time
0,0.702902,0.174995,0.218956,00:01


In [ ]:
#| export
class SeperateTCN(nn.Module):
    def __init__(self,input_list,output_size,hl_depth=1,hl_width=10,act = Mish,bn=False,stateful=False,final_layer=3):
        super().__init__()
        self.input_list =np.cumsum([0] + input_list)
        
        tcn_width = hl_width//len(input_list)
        layers = [ [TCN_Block(n if i==0 else tcn_width,tcn_width,
                                      dilation=2**(i),bn=bn,activation=act)
                                          for i in range(hl_depth)]
                        for n in input_list]
        self.layers = nn.ModuleList([nn.Sequential(*l) for l in layers])
        
        self.rec_field = (2**hl_depth)-1
        self.final = SeqLinear(tcn_width*len(input_list),output_size,hidden_size=hl_width,hidden_layer=final_layer)
        self.x_init = None
        self.stateful = stateful
        
    def forward(self, x):      
        if self.x_init is not None:
            if self.x_init.shape[0] != x.shape[0]: 
                self.x_init = None
            elif self.stateful: 
                x = torch.cat([self.x_init,x],dim=1)
        
        tcn_out = [layer(x[...,self.input_list[i]:self.input_list[i+1]].transpose(1,2)) 
                   for i,layer in enumerate(self.layers)]
        out = torch.cat(tcn_out,dim=1).transpose(1,2)
        
        out = self.final(out)
        
        if self.stateful:
            if self.x_init is not None: out = out[:,self.rec_field:]
            self.x_init = x[:,-self.rec_field:]
            
        return out
    
    def reset_state(self):
        self.x_init = None

## CRNNs

In [ ]:
#| export
class CRNN(nn.Module):
    def __init__(self,input_size,output_size,num_ft=10,num_cnn_layers=4,num_rnn_layers=2,hs_cnn=10,hs_rnn=10,
         hidden_p=0, input_p=0, weight_p=0, rnn_type='gru',stateful=False):
        super().__init__()
        self.cnn = TCN(input_size,num_ft,num_cnn_layers,hs_cnn,act=nn.ReLU,stateful=stateful)
        self.rnn = SimpleRNN(num_ft,output_size,num_layers=num_rnn_layers,hidden_size=hs_rnn,
                       hidden_p=hidden_p, input_p=input_p, weight_p=weight_p, 
                        rnn_type=rnn_type,stateful=stateful)
        
    def forward(self, x):
        return self.rnn(self.cnn(x))
    

In [ ]:
model = CRNN(1,1,10)
lrn = Learner(dls,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,0.083012,0.067571,00:03


In [ ]:
model = CRNN(1,1,10,rnn_type='gru')
lrn = Learner(dls,model,loss_func=nn.MSELoss()).fit(1)

epoch,train_loss,valid_loss,time
0,0.087968,0.066237,00:02


### CRNN Learner


In [ ]:
#| export
@delegates(CRNN, keep=True)
def CRNNLearner(dls,loss_func=nn.L1Loss(),metrics=[fun_rmse],n_skip=0,opt_func=Adam,cbs=None,**kwargs):
    inp,out = get_inp_out_size(dls)
    model = CRNN(inp,out,**kwargs)
  
    skip = partial(SkipNLoss,n_skip=n_skip)
        
    metrics= [skip(f) for f in metrics]
    loss_func = skip(loss_func)
        
    lrn = Learner(dls,model,loss_func=loss_func,opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn


In [ ]:
CRNNLearner(dls_pred,rnn_type='gru').fit(1,3e-2)


epoch,train_loss,valid_loss,fun_rmse,time
0,0.211474,0.123397,0.155189,00:00


### Autoregressive TCN Learner


In [ ]:
#| export
@delegates(TCN, keep=True)
def AR_TCNLearner(dls,hl_depth=3,alpha=1,beta=1,early_stop=0,metrics=None,n_skip=None,opt_func=Adam,**kwargs):
    n_skip = 2**hl_depth if n_skip is None else n_skip
    skip = partial(SkipNLoss,n_skip=n_skip)
    
    inp,out = get_inp_out_size(dls)
    model = AR_Model(TCN(inp+out,out,hl_depth,**kwargs),ar=False)
    model.init_normalize(dls.one_batch())
    
    cbs=[ARInitCB(),TimeSeriesRegularizer(alpha=alpha,beta=beta,modules=[model.model.conv_layers[-1]])]#SaveModelCallback()
    if early_stop > 0:
        cbs += [EarlyStoppingCallback(patience=early_stop)]
        
    if metrics is None: metrics=SkipNLoss(fun_rmse,n_skip)
        
    lrn = Learner(dls,model,loss_func=nn.L1Loss(),opt_func=opt_func,metrics=metrics,cbs=cbs,lr=3e-3)
    return lrn


In [ ]:
AR_TCNLearner(dls_pred).fit(1)


epoch,train_loss,valid_loss,fun_rmse,time
0,0.249652,0.242456,0.302200,00:00


In [ ]:
#| export
class SeperateCRNN(nn.Module):
    def __init__(self,input_list,output_size,num_ft=10,num_cnn_layers=4,num_rnn_layers=2,hs_cnn=10,hs_rnn=10,
         hidden_p=0, input_p=0, weight_p=0, rnn_type='gru',stateful=False):
        super().__init__()
        self.cnn = SeperateTCN(input_list,num_ft,num_cnn_layers,hs_cnn,act=nn.ReLU,stateful=stateful,final_layer=0)
        self.rnn = SimpleRNN(num_ft,output_size,num_layers=num_rnn_layers,hidden_size=hs_rnn,
                       hidden_p=hidden_p, input_p=input_p, weight_p=weight_p, 
                        rnn_type=rnn_type,stateful=stateful)
        
    def forward(self, x):
        return self.rnn(self.cnn(x))

In [ ]:
#| include: false
import nbdev; nbdev.nbdev_export()